In [1]:
%autosave 180
%load_ext autoreload
%autoreload 2
%matplotlib tk

#|
import numpy as np
import matplotlib.pyplot as plt



Autosaving every 180 seconds


In [15]:
##########################################################
########## COMPUTE VELOCITY FROM ROTARY ENCODER ##########
##########################################################
from tqdm import trange

#
def binarize(data):
    
    idx = np.where(data<2.5)[0]
    idx2 = np.where(data>=2.5)[0]
    
    data[idx] = 0
    data[idx2] = 1
    
    #
    return data

#
def get_velocity(data):
    
    # this contains the absolute time stamps of the rotary encoder readings
    try:
        abs_times_ttl_read = data['abs_times_ttl_read']
    except:
        print ("missing abs_times_ttl_read; using abs_times")
        abs_times_ttl_read = data['abs_times']
    print ("lick abs_times_ttl_read: ", abs_times_ttl_read.shape, abs_times_ttl_read)

    
    # lad the rotary encoder data
    rot1 = data['rotary_encoder1_abstime']
    rot2 = data['rotary_encoder2_abstime']
    print ("rot2: ", rot2.shape)
    #abs_times_ttl_read = data['abs_times_ttl_read']
    #print ("abs_times_ttl_read: ", abs_times_ttl_read.shape)

    # distance
    n_clicks_per_rotation = 500
    ball_diameter = 0.2  # distance in meters
    ball_circumference = ball_diameter*3.141592
    dist_per_click = ball_circumference/n_clicks_per_rotation
    
    # time
    sample_rate = 1000
    seconds_per_time_stamp = 1/sample_rate
        
    # detect the clicks
    bin1 = binarize(rot1)
    bin2 = binarize(rot2)
    
    #
    clicks = np.array((bin1, bin2)).T.squeeze()
    print ("Clicks: ", clicks.shape, clicks[:10])
    print ("abs_times_ttl_read: ", abs_times_ttl_read.shape, abs_times_ttl_read[:10], abs_times_ttl_read[-10:])
        
    #
    vel = []
    times = []
    time_since_last_click = 0
    rot1_last_state = clicks[0,0]
    for k in trange(0,clicks.shape[0],1):
        
        # check if the state of the rotary encoder changed
        if clicks[k,0]!=rot1_last_state:
            distance = dist_per_click  #only walked 1 click

            # check to see how long since last time point
            time = time_since_last_click*seconds_per_time_stamp
            
            # compute velocity
            v = distance/time 
            
            #
            vel.append(v)
            times.append(abs_times_ttl_read[k])
            #times.append(k/sample_rate)       
            
            #
            time_since_last_click=0
            last_click_location=k #.copy()
            rot1_last_state=clicks[k,0]

        # add a zero velocity if no click
        #else:
        #    vel.append(0)
        #    times.append(k/sample_rate)


        #    
        time_since_last_click+=1

    #
    times = np.array(times)
    
    # ok, now find the nearest value in times to the abs_times_ca_read
    try:
        abs_times_ca_read = data['abs_times_ca_read']
    except:
        print ("missing abs_times_ca_read; using abs_times")
        abs_times_ca_read = data['ttl_times']
    print ("lick abs_times_ca_read: ", abs_times_ca_read.shape, abs_times_ca_read)

    times_ttl_final = []
    velocity_final = []
    for k in trange(abs_times_ca_read.shape[0]):
        idx = np.argmin(np.abs(times-abs_times_ca_read[k]))
        #print (idx, times[idx], abs_times_ca_read[k])
        times_ttl_final.append(times[idx])
        velocity_final.append(vel[idx])
    
    times_ttl_final = np.array(times_ttl_final)
    velocity_final = np.array(velocity_final)

    #
    reward_times = data['reward_times'].squeeze().T[:,1]

    #
    return velocity_final, reward_times
    

##################################################################
##################################################################
##################################################################
fname = '/media/cat/8TB/donato/bmi/DON-012502/20230130/results.npz'
data = np.load(fname,
               allow_pickle=True)

#
velocities, reward_times = get_velocity(data)

# 
print ("velocity array has # of entries", velocities.shape)

#
plt.figure()
plt.plot(velocities)
plt.xlabel("TTL pulse #")
plt.ylabel("Velocity (m/s)")

# add reward times 
for k in range(reward_times.shape[0]):
    # use vertical lines from 0 to 1
    plt.plot([reward_times[k], reward_times[k]], [0, 1], 'r', 
             # line style dashed
             linestyle='--',)

#
plt.show()

missing abs_times_ttl_read; using abs_times
lick abs_times_ttl_read:  (3006991,) [  18.8382939   18.8385028   18.8386601 ... 3023.7724147 3023.772614
 3023.7743895]
rot2:  (3006991, 1)
Clicks:  (3006991, 2) [[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
abs_times_ttl_read:  (3006991,) [18.8382939 18.8385028 18.8386601 18.8388132 18.8389673 18.8391181
 18.83927   18.8394198 18.8395701 18.8397185] [3023.7646294 3023.7664009 3023.7666185 3023.7684526 3023.7686543
 3023.7704687 3023.7708555 3023.7724147 3023.772614  3023.7743895]


100%|██████████| 3006991/3006991 [00:00<00:00, 3433102.80it/s]


missing abs_times_ca_read; using abs_times
lick abs_times_ca_read:  (89951,) [  23.8167937   23.8505776   23.8828276 ... 3023.7069419 3023.7404994
 3023.7743895]


100%|██████████| 89951/89951 [00:25<00:00, 3572.54it/s]


velocity array has # of entries (89951,)
